# 특징 생성

- 범주 인코딩 (Nominal, Ordinal 형식) -> One-hot Encoding
- 결합 (Add합계, Divide평균, Subtract편차  //  Multiply) (파생변수 만들 때 사용)
- 분해 (변수 분해)
- 차원 축소 (Principal Component Analysis, PCA, 주성분 분석)

#### Principal Component Analysis, PCA, 주성분 분석
- 원본 데이터의 분산 기반 특징을 생성
- 주성분 간 서로 독립을 이루도록 구성 (상관관계가 없도록 구성)

#### 차원 축소 목적 특징 생성
- 군집 결과를 추후 분류/회귀 등 문제해결을 위한 입력변수로 활용함 (stacking 방법)
- 여러개의 특징을 하나의 특징으로 축소 -> 모델 연산 비용 감소 추구

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# One-hot Encoding
df = pd.get_dummies(df, columns=['city'])
df.head()

In [ ]:
# 범주형 date 컬럼 -> datetime 형식 변환
df['date'] = pd.to_datetime(df['date'])
print(df.head())

df['year'] = df['date'].dt.year # 연도
df['month'] = df['date'].dt.month # 월
df['day'] = df['date'].dt.day # 일
df['hour'] = df['date'].dt.hour # 시간
df['dayofweek'] = df['date'].dt.dayofweek # 요일 (월 = 0)
print(df.head())

df['ampm'] = 'AM'
df.loc[df['hour'] > 12, 'ampm'] = 'PM'
print(df.head())

### PCA, 주성분 분석

In [ ]:
# 차원 축소 기반 특징 생성 - PCA (주성분분석)

# input 및 target 구분
input_df = df.drop(['col1'], axis=1) # 열 추출
target_df = df[['col1']]

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler

# 각 변수의 스케일이 다른 것을 표준화 수행
std_scaler.fit(input_df)
input_scaled = std_scaler.transform(input_df)
print(input_scaled)

pca = PCA(n_components=2)
pca.fit(input_scaled)
X_pca = pca.transform(input_scaled)
print(X_pca)

# 주성분 분석으로 변수들을 2개(pc1, pc2)로 축소시킴
X_pca_df = pd.DataFrame(X_pca, columns = ['pc1', 'pc2'])
display(X_pca_df)

In [ ]:
# 2개의 주성분 분석을 통한 변수와 진단결과를 비교
target_df = target_df.reset_index()
pca_df = pd.merge(X_pca_df, target_df, left_index=True, right_index=True, how='inner')
pca_df = pca_df[['pc1', 'pc2', 'diagnosis']]
display(pca_df)

In [ ]:
# 산점도 그래프로 표현
ax = sns.scatterplot(x='pc1', y='pc2', hue='diagnosis', data=pca_df, palette=['green', 'red'])

### 군집 분석

In [ ]:
from sklearn.cluster import KMeans


subset_df = input_df.iloc[:, 0:15]
display(subset_df)

# 데이터 스케일링
std_scaler.fit(subset_df)
subset_input_scaled = std_scaler.transform(subset_df)
print(subset_input_scaled)

# K-means 클러스터링 
k = 5
model = KMeans(n_cluster = k, random_state = 10)
model.fit(subset_input_scaled)
target_df['cluster'] = model.fit_predict(subset_input_scaled)
display(target_df)

# 15 개의 임의의 변수로 만들어진 하나의 특징과 기존 Target 변수 비교
pd.crosstab(target_df.diagnosis, target_df.cluster)